### Lab2 (a) Model preperation

In [1]:
from resnet20 import ResNetCIFAR
from train_util import train, finetune, test
import torch
import numpy as np

import time

import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
import torch.optim as optim

from FP_layers import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [5]:
net = ResNetCIFAR(num_layers=20, Nbits=None)
net = net.to(device)

In [3]:
# Load the best weight paramters
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)

Files already downloaded and verified
Test Loss=0.3231, Test accuracy=0.9151


### Lab2 (b) Prune by percentage

In [4]:
def prune_by_percentage(layer, q=70.0, device = device):
    """
    Pruning the weight paramters by threshold.
    :param q: pruning percentile. 'q' percent of the least 
    significant weight parameters will be pruned.
    """
    # Convert the weight of "layer" to numpy array
    weights = layer.weight.detach().cpu().numpy()
    
    # Compute the q-th percentile of the abs of the converted array
    qth_percentile = np.percentile(np.abs(weights), q)

    # Generate a binary mask same shape as weight to decide which element to prune
    mask = np.where(np.abs(weights) < qth_percentile, 0, np.abs(weights)) #Question: anything smaller than qth_percentile should be pruned right?? 
    mask = np.where(mask != 0, 1, mask)

    # Convert mask to torch tensor and put on GPU
    mask = torch.from_numpy(mask).to(device)

    # Multiply the weight by mask to perform pruning
    layer.weight.data = layer.weight.data * mask
    
    return mask
    

In [176]:
#when q = 0.3

net.load_state_dict(torch.load("pretrained_model.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # change q value
        prune_by_percentage(layer, q=30.0)
        
        # Optional: Check the sparsity you achieve in each layer
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.detach().cpu().numpy()
        # Count number of zeros
        zeros = np.count_nonzero(np_weight == 0)
        # Count number of parameters
        total = np_weight.size
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))
        
test(net)

Sparsity of head_conv.0.conv: 0.30092592592592593
Sparsity of body_op.0.conv1.0.conv: 0.2999131944444444
Sparsity of body_op.0.conv2.0.conv: 0.2999131944444444
Sparsity of body_op.1.conv1.0.conv: 0.2999131944444444
Sparsity of body_op.1.conv2.0.conv: 0.2999131944444444
Sparsity of body_op.2.conv1.0.conv: 0.2999131944444444
Sparsity of body_op.2.conv2.0.conv: 0.2999131944444444
Sparsity of body_op.3.conv1.0.conv: 0.3001302083333333
Sparsity of body_op.3.conv2.0.conv: 0.3000217013888889
Sparsity of body_op.4.conv1.0.conv: 0.3000217013888889
Sparsity of body_op.4.conv2.0.conv: 0.3000217013888889
Sparsity of body_op.5.conv1.0.conv: 0.3000217013888889
Sparsity of body_op.5.conv2.0.conv: 0.3000217013888889
Sparsity of body_op.6.conv1.0.conv: 0.3000217013888889
Sparsity of body_op.6.conv2.0.conv: 0.2999945746527778
Sparsity of body_op.7.conv1.0.conv: 0.2999945746527778
Sparsity of body_op.7.conv2.0.conv: 0.2999945746527778
Sparsity of body_op.8.conv1.0.conv: 0.2999945746527778
Sparsity of bod

In [154]:
#when q = 0.5

net.load_state_dict(torch.load("pretrained_model.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # change q value
        prune_by_percentage(layer, q=50.0)
        
        # Optional: Check the sparsity you achieve in each layer
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.detach().cpu().numpy()
        # Count number of zeros
        zeros = np.count_nonzero(np_weight == 0)
        # Count number of parameters
        total = np_weight.size
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))
        
test(net)

Sparsity of head_conv.0.conv: 0.5
Sparsity of body_op.0.conv1.0.conv: 0.5
Sparsity of body_op.0.conv2.0.conv: 0.5
Sparsity of body_op.1.conv1.0.conv: 0.5
Sparsity of body_op.1.conv2.0.conv: 0.5
Sparsity of body_op.2.conv1.0.conv: 0.5
Sparsity of body_op.2.conv2.0.conv: 0.5
Sparsity of body_op.3.conv1.0.conv: 0.5
Sparsity of body_op.3.conv2.0.conv: 0.5
Sparsity of body_op.4.conv1.0.conv: 0.5
Sparsity of body_op.4.conv2.0.conv: 0.5
Sparsity of body_op.5.conv1.0.conv: 0.5
Sparsity of body_op.5.conv2.0.conv: 0.5
Sparsity of body_op.6.conv1.0.conv: 0.5
Sparsity of body_op.6.conv2.0.conv: 0.5
Sparsity of body_op.7.conv1.0.conv: 0.5
Sparsity of body_op.7.conv2.0.conv: 0.5
Sparsity of body_op.8.conv1.0.conv: 0.5
Sparsity of body_op.8.conv2.0.conv: 0.5
Sparsity of final_fc.linear: 0.5
Files already downloaded and verified
Test Loss=0.6774, Test accuracy=0.8210


In [155]:
#when q = 0.7

net.load_state_dict(torch.load("pretrained_model.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # change q value
        prune_by_percentage(layer, q=70.0)
        
        # Optional: Check the sparsity you achieve in each layer
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.detach().cpu().numpy()
        # Count number of zeros
        zeros = np.count_nonzero(np_weight == 0)
        # Count number of parameters
        total = np_weight.size
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))
        
test(net)

Sparsity of head_conv.0.conv: 0.6990740740740741
Sparsity of body_op.0.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.0.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.1.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.1.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.2.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.2.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.3.conv1.0.conv: 0.6998697916666666
Sparsity of body_op.3.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.4.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.4.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.5.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.5.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.6.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.6.conv2.0.conv: 0.7000054253472222
Sparsity of body_op.7.conv1.0.conv: 0.7000054253472222
Sparsity of body_op.7.conv2.0.conv: 0.7000054253472222
Sparsity of body_op.8.conv1.0.conv: 0.7000054253472222
Sparsity of body

### Lab2 (c) Finetune pruned model

In [5]:
def finetune_after_prune(net, trainloader, criterion, optimizer, prune=True, global_pruning = False):
    """
    Finetune the pruned model for a single epoch
    Make sure pruned weights are kept as zero
    """
    # Build a dictionary for the nonzero weights
    weight_mask = {}
    
    if global_pruning:
        weight_mask = global_prune_by_percentage(net)
    else:
        for name,layer in net.named_modules():
            if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
                # Your code here: generate a mask in GPU torch tensor to have 1 for nonzero element and 0 for zero element
                weight_mask[name] = prune_by_percentage(layer)
    
    global_steps = 0
    train_loss = 0
    correct = 0
    total = 0
    start = time.time()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = net(inputs)
        optimizer.zero_grad()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        if prune:
            for name,layer in net.named_modules():
                if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
                    # Your code here: Use weight_mask to make sure zero elements remains zero
                    layer.weight.data = layer.weight.data * weight_mask[name]
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        global_steps += 1

        if global_steps % 50 == 0:
            end = time.time()
            batch_size = 256
            num_examples_per_second = 50 * batch_size / (end - start)
            print("[Step=%d]\tLoss=%.4f\tacc=%.4f\t%.1f examples/second"
                 % (global_steps, train_loss / (batch_idx + 1), (correct / total), num_examples_per_second))
            start = time.time()
    
    

In [15]:
# Get pruned model
net.load_state_dict(torch.load("pretrained_model.pt"))
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        prune_by_percentage(layer, q=70.0)

# Training setup, do not change
batch_size=256
lr=0.002
reg=1e-4

print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.875, weight_decay=reg, nesterov=False)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [16]:
# Model finetuning
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    net.train()
    finetune_after_prune(net, trainloader, criterion, optimizer)
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))

    if val_acc > best_acc:
        best_acc = val_acc
        print("Saving...")
        torch.save(net.state_dict(), "net_after_finetune.pt")


Epoch: 0


[Step=50]	Loss=0.4009	acc=0.8620	6673.4 examples/second
[Step=100]	Loss=0.3559	acc=0.8766	7572.0 examples/second
[Step=150]	Loss=0.3334	acc=0.8849	7745.7 examples/second
Test Loss=0.4242, Test acc=0.8682
Saving...

Epoch: 1
[Step=50]	Loss=0.2407	acc=0.9161	6385.9 examples/second
[Step=100]	Loss=0.2427	acc=0.9151	7439.5 examples/second
[Step=150]	Loss=0.2375	acc=0.9176	7591.7 examples/second
Test Loss=0.3937, Test acc=0.8771
Saving...

Epoch: 2
[Step=50]	Loss=0.2249	acc=0.9197	6591.4 examples/second
[Step=100]	Loss=0.2194	acc=0.9228	7728.2 examples/second
[Step=150]	Loss=0.2154	acc=0.9251	7106.3 examples/second
Test Loss=0.3778, Test acc=0.8808
Saving...

Epoch: 3
[Step=50]	Loss=0.1917	acc=0.9341	6720.1 examples/second
[Step=100]	Loss=0.1950	acc=0.9328	7716.4 examples/second
[Step=150]	Loss=0.1975	acc=0.9318	7505.5 examples/second
Test Loss=0.3690, Test acc=0.8820
Saving...

Epoch: 4
[Step=50]	Loss=0.1977	acc=0.9303	6248.4 examples/second
[Step=100]	Loss=0.1940	acc=0.9325	7241.3 example

In [17]:
# Check sparsity of the finetuned model, make sure it's not changed
net.load_state_dict(torch.load("net_after_finetune.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.detach().cpu().numpy()
        # Count number of zeros
        zeros = np.count_nonzero(np_weight == 0)
        # Count number of parameters
        total = np_weight.size
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))
        
test(net)

Sparsity of head_conv.0.conv: 0.6990740740740741
Sparsity of body_op.0.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.0.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.1.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.1.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.2.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.2.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.3.conv1.0.conv: 0.6998697916666666
Sparsity of body_op.3.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.4.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.4.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.5.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.5.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.6.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.6.conv2.0.conv: 0.7000054253472222
Sparsity of body_op.7.conv1.0.conv: 0.7000054253472222
Sparsity of body_op.7.conv2.0.conv: 0.7000054253472222
Sparsity of body_op.8.conv1.0.conv: 0.7000054253472222
Sparsity of body

### Lab2 (d) Iterative pruning

In [18]:
net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    
    net.train()
    if epoch<10:
        for name,layer in net.named_modules():
            if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
                # Increase model sparsity
                q = 7 * (epoch + 1) # Question: is it epoch or epoch + 1?
                prune_by_percentage(layer, q=q)
    print("Pruning q: ", q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)
    
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))
    
    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "net_after_iterative_prune.pt")


Epoch: 0
Pruning q:  7


[Step=50]	Loss=0.3627	acc=0.8770	6751.9 examples/second
[Step=100]	Loss=0.3127	acc=0.8925	7704.8 examples/second
[Step=150]	Loss=0.2850	acc=0.9029	7581.2 examples/second
Test Loss=0.3843, Test acc=0.8776

Epoch: 1
Pruning q:  14
[Step=50]	Loss=0.2657	acc=0.9106	6071.2 examples/second
[Step=100]	Loss=0.2478	acc=0.9155	6748.5 examples/second
[Step=150]	Loss=0.2328	acc=0.9199	7451.6 examples/second
Test Loss=0.3711, Test acc=0.8828

Epoch: 2
Pruning q:  21
[Step=50]	Loss=0.2245	acc=0.9224	6580.1 examples/second
[Step=100]	Loss=0.2190	acc=0.9234	7636.9 examples/second
[Step=150]	Loss=0.2128	acc=0.9261	7573.4 examples/second
Test Loss=0.3675, Test acc=0.8853

Epoch: 3
Pruning q:  28
[Step=50]	Loss=0.2126	acc=0.9263	6561.1 examples/second
[Step=100]	Loss=0.2056	acc=0.9291	7504.7 examples/second
[Step=150]	Loss=0.1995	acc=0.9310	7515.7 examples/second
Test Loss=0.3591, Test acc=0.8854

Epoch: 4
Pruning q:  35
[Step=50]	Loss=0.2057	acc=0.9274	6522.6 examples/second
[Step=100]	Loss=0.2011	acc=0

In [20]:
# Check sparsity of the final model, make sure it's 70%
net.load_state_dict(torch.load("net_after_iterative_prune.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here: can copy from previous question
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.detach().cpu().numpy()
        # Count number of zeros
        zeros = np.count_nonzero(np_weight == 0)
        # Count number of parameters
        total = np_weight.size
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))
        
test(net)

Sparsity of head_conv.0.conv: 0.6990740740740741
Sparsity of body_op.0.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.0.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.1.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.1.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.2.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.2.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.3.conv1.0.conv: 0.6998697916666666
Sparsity of body_op.3.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.4.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.4.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.5.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.5.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.6.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.6.conv2.0.conv: 0.7000054253472222
Sparsity of body_op.7.conv1.0.conv: 0.7000054253472222
Sparsity of body_op.7.conv2.0.conv: 0.7000054253472222
Sparsity of body_op.8.conv1.0.conv: 0.7000054253472222
Sparsity of body

### Lab2 (e) Global iterative pruning

In [7]:
def global_prune_by_percentage(net, q=70.0):
    """
    Pruning the weight paramters by threshold.
    :param q: pruning percentile. 'q' percent of the least 
    significant weight parameters will be pruned.
    """
    # A list to gather all the weights
    flattened_weights = []
    # Find global pruning threshold
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            weights = layer.weight.detach().cpu().numpy()
            # Flatten the weight and append to flattened_weights
            flattened_weights.append(weights.flatten())
    
    # Concate all weights into a np array
    flattened_weights = np.concatenate(flattened_weights)
    # Find global pruning threshold
    thres = np.percentile(np.abs(flattened_weights), q)
    
    # Apply pruning threshold to all layers
    mask_weights = {}
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            weights = layer.weight.detach().cpu().numpy()
            
            # Generate a binary mask same shape as weight to decide which element to prune
            mask = np.where(np.abs(weights) < thres, 0, np.abs(weights)) #Question: anything smaller than q_th percentile is pruned right?
            mask = np.where(mask != 0, 1, mask)

            # Convert mask to torch tensor and put on GPU
            mask = torch.from_numpy(mask).to(device)
            mask_weights[name] = mask
            
            # Multiply the weight by mask to perform pruning
            layer.weight.data = layer.weight.data * mask
            
    return mask_weights            

In [8]:
net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    
    net.train()
    # Increase model sparsity
    if epoch<10:
        q=(epoch+1)*7 #question: shouldn't this be *7 instead of *8?? I changed this to 7. Also moved locations for this.
        global_prune_by_percentage(net, q=q)
    print("q %: ", q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False, global_pruning=True)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer, global_pruning = True)
    
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))
    
    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "net_after_global_iterative_prune.pt")


Epoch: 0
q %:  7


[Step=50]	Loss=0.2202	acc=0.9234	5427.0 examples/second
[Step=100]	Loss=0.2039	acc=0.9293	7284.7 examples/second
[Step=150]	Loss=0.1930	acc=0.9332	7872.5 examples/second
Test Loss=0.3407, Test acc=0.8899

Epoch: 1
q %:  14
[Step=50]	Loss=0.1914	acc=0.9342	6742.3 examples/second
[Step=100]	Loss=0.1811	acc=0.9376	7589.4 examples/second
[Step=150]	Loss=0.1750	acc=0.9390	7682.1 examples/second
Test Loss=0.3359, Test acc=0.8924

Epoch: 2
q %:  21
[Step=50]	Loss=0.1711	acc=0.9421	6256.6 examples/second
[Step=100]	Loss=0.1632	acc=0.9449	7700.6 examples/second
[Step=150]	Loss=0.1623	acc=0.9439	7190.0 examples/second
Test Loss=0.3325, Test acc=0.8937

Epoch: 3
q %:  28
[Step=50]	Loss=0.1712	acc=0.9395	6336.4 examples/second
[Step=100]	Loss=0.1626	acc=0.9427	7721.0 examples/second
[Step=150]	Loss=0.1583	acc=0.9445	7575.9 examples/second
Test Loss=0.3271, Test acc=0.8957

Epoch: 4
q %:  35
[Step=50]	Loss=0.1494	acc=0.9486	6455.1 examples/second
[Step=100]	Loss=0.1494	acc=0.9487	7679.3 examples/se

In [10]:
net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))

zeros_sum = 0
total_sum = 0
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.detach().cpu().numpy()
        # Count number of zeros
        zeros = np.count_nonzero(np_weight == 0)
        # Count number of parameters
        total = np_weight.size
        zeros_sum+=zeros
        total_sum+=total
        print('Sparsity of '+name+': '+str(zeros/total))
print('Total sparsity of: '+str(zeros_sum/total_sum))
test(net)

Sparsity of head_conv.0.conv: 0.24305555555555555
Sparsity of body_op.0.conv1.0.conv: 0.5512152777777778
Sparsity of body_op.0.conv2.0.conv: 0.5269097222222222
Sparsity of body_op.1.conv1.0.conv: 0.5199652777777778
Sparsity of body_op.1.conv2.0.conv: 0.5529513888888888
Sparsity of body_op.2.conv1.0.conv: 0.5186631944444444
Sparsity of body_op.2.conv2.0.conv: 0.5655381944444444
Sparsity of body_op.3.conv1.0.conv: 0.5251736111111112
Sparsity of body_op.3.conv2.0.conv: 0.5832248263888888
Sparsity of body_op.4.conv1.0.conv: 0.6157769097222222
Sparsity of body_op.4.conv2.0.conv: 0.6769748263888888
Sparsity of body_op.5.conv1.0.conv: 0.6119791666666666
Sparsity of body_op.5.conv2.0.conv: 0.7030164930555556
Sparsity of body_op.6.conv1.0.conv: 0.6143120659722222
Sparsity of body_op.6.conv2.0.conv: 0.6512586805555556
Sparsity of body_op.7.conv1.0.conv: 0.66259765625
Sparsity of body_op.7.conv2.0.conv: 0.7189127604166666
Sparsity of body_op.8.conv1.0.conv: 0.7477213541666666
Sparsity of body_op.

### Lab 3 (b) and (c): Fixed-point quantization

b)

In [2]:
# Define quantized model and load weight

all_nbits = [6, 5, 4, 3, 2]

for Nbits in all_nbits:
    print("Nbits: ", Nbits)
    net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
    net = net.to(device)
    net.load_state_dict(torch.load("pretrained_model.pt"))
    test(net)
    print("-"*50)

Nbits:  6
Files already downloaded and verified
Test Loss=0.3364, Test accuracy=0.9145
--------------------------------------------------
Nbits:  5
Files already downloaded and verified
Test Loss=0.3390, Test accuracy=0.9112
--------------------------------------------------
Nbits:  4
Files already downloaded and verified
Test Loss=0.3861, Test accuracy=0.8972
--------------------------------------------------
Nbits:  3
Files already downloaded and verified
Test Loss=0.9874, Test accuracy=0.7662
--------------------------------------------------
Nbits:  2
Files already downloaded and verified
Test Loss=9.5441, Test accuracy=0.0899
--------------------------------------------------


c)

In [2]:
all_nbits = [4, 3, 2]

for Nbits in all_nbits:
    print("Nbits: ", Nbits)
    net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
    net = net.to(device)
    net.load_state_dict(torch.load("pretrained_model.pt"))
    # Quantized model finetuning
    finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)
    # Load the model with best accuracy
    net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
    test(net)   
    print("-"*50)

Nbits:  4
==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=0.0734	acc=0.9753	3210.5 examples/second
[Step=100]	Loss=0.0686	acc=0.9764	6659.8 examples/second
[Step=150]	Loss=0.0652	acc=0.9775	6591.1 examples/second
Test Loss=0.3376, Test acc=0.9091
Saving...

Epoch: 1
[Step=200]	Loss=0.0741	acc=0.9775	3235.1 examples/second
[Step=250]	Loss=0.0655	acc=0.9766	6534.6 examples/second
[Step=300]	Loss=0.0635	acc=0.9778	6713.8 examples/second
[Step=350]	Loss=0.0639	acc=0.9777	6621.2 examples/second
Test Loss=0.3338, Test acc=0.9097
Saving...

Epoch: 2
[Step=400]	Loss=0.0508	acc=0.9854	3229.2 examples/second
[Step=450]	Loss=0.0573	acc=0.9802	6428.4 examples/second
[Step=500]	Loss=0.0597	acc=0.9791	6726.7 examples/second
[Step=550]	Loss=0.0606	acc=0.9793	6583.4 examples/second
Test Loss=0.3311, Test acc=0.9114
Saving...

Epoch: 3
[Step=600]	Loss=0.0615	acc=0.9766	3241.8 examples/second
[Step=650]	Loss=0.0611	acc=0.9796	6571

### Lab3 (d) Quantize pruned model

In [3]:
all_nbits = [4, 3, 2]

for Nbits in all_nbits:
    print("Nbits: ", Nbits)
    net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
    net = net.to(device)
    net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))
    print("Loss and Test Accuracy Before Quantization")
    test(net)
    # Quantized model finetuning
    finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)
    # Load the model with best accuracy
    net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
    test(net)
    print("-"*50)

Nbits:  4
Loss and Test Accuracy Before Quantization
Files already downloaded and verified
Test Loss=0.3653, Test accuracy=0.8848
==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=0.7158	acc=0.8113	5923.7 examples/second
[Step=100]	Loss=0.5570	acc=0.8462	6674.0 examples/second
[Step=150]	Loss=0.4679	acc=0.8662	6353.9 examples/second
Test Loss=0.3831, Test acc=0.8753
Saving...

Epoch: 1
[Step=200]	Loss=0.2644	acc=0.9033	3214.2 examples/second
[Step=250]	Loss=0.2307	acc=0.9209	6402.6 examples/second
[Step=300]	Loss=0.2253	acc=0.9247	6646.6 examples/second
[Step=350]	Loss=0.2219	acc=0.9256	6520.7 examples/second
Test Loss=0.4127, Test acc=0.8756
Saving...

Epoch: 2
[Step=400]	Loss=0.2147	acc=0.9355	3194.2 examples/second
[Step=450]	Loss=0.2112	acc=0.9285	6553.2 examples/second
[Step=500]	Loss=0.2091	acc=0.9297	6566.1 examples/second
[Step=550]	Loss=0.2040	acc=0.9303	6629.2 examples/second
Test Loss=0.3677, Test acc=0.

### Lab3 (e) Symmetric quantization
#### Implement symmetric quantization in FP_layers.py, and repeat the process in (b)

In [2]:
# check the performance of symmetric quantization with 6, 5, 4, 3, 2 bits

# Define quantized model and load weight

all_nbits = [6, 5, 4, 3, 2]

for Nbits in all_nbits:
    print("Nbits: ", Nbits)
    net = ResNetCIFAR(num_layers=20, Nbits=Nbits, symmetric = True)
    net = net.to(device)
    net.load_state_dict(torch.load("pretrained_model.pt"))
    test(net)
    print("-"*50)

Nbits:  6
Files already downloaded and verified
Test Loss=0.3276, Test accuracy=0.9133
--------------------------------------------------
Nbits:  5
Files already downloaded and verified
Test Loss=0.3418, Test accuracy=0.9071
--------------------------------------------------
Nbits:  4
Files already downloaded and verified
Test Loss=0.5689, Test accuracy=0.8531
--------------------------------------------------
Nbits:  3
Files already downloaded and verified
Test Loss=1.2511, Test accuracy=0.7151
--------------------------------------------------
Nbits:  2
Files already downloaded and verified
Test Loss=96.5350, Test accuracy=0.1000
--------------------------------------------------
